In [6]:
pip uninstall -y pinecone pinecone-client

Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip cache purge

Files removed: 542
Note: you may need to restart the kernel to use updated packages.


In [1]:
! pip install pinecone

! pip install llama-index
! pip install llama-index-vector-stores-pinecone
! pip install llama-index-embeddings-cohere
! pip install llama-index-llms-cohere


Defaulting to user installation because normal site-packages is not writeable
  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 6.9 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-pinecone 0.7.0 requires llama-index-core<0.14,>=0.13.0, but you have llama-index-core 0.12.52.post1 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.52.post1
    Uninstalling llama-index-core-0.12.52.post1:
      Successfully uninstalled llama-index-core-0.12.52.post1
ERROR: pip's dependency resolver does not currently take into 

In [3]:
! pip install --upgrade pinecone
! pip install --upgrade llama-index-vector-stores-pinecone

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached llama_index_core-0.13.3-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.13.3-py3-none-any.whl (7.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.52.post1
    Uninstalling llama-index-core-0.12.52.post1:
      Successfully uninstalled llama-index-core-0.12.52.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-cohere 0.5.0 requires llama-index-core<0.13,>=0.12.0, but you have llama-index-core 0.13.3 which is incompatible.


In [7]:
import os
import pinecone
import cohere
import time
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.llms.cohere import Cohere
from tqdm import tqdm

# --- 1. CONFIGURAÇÕES INICIAIS (sem mudanças) ---
PINECONE_API_KEY = "pcsk_5RSTcB_8GfPp22T2dhbtGz59ihsbPZQnG6HHnia2QJosnBy22T8ZTu1wdcwhqdWqhRrUcm"
COHERE_API_KEY = "RAjqpE9uTVFfTTaF0RIGUM4PaKBeQ7insNIu9KLC"
INDEX_NAME = "teste"
PDF_FOLDER_PATH = "./documentos"


# --- Constantes para o controle de lote ---
BATCH_SIZE = 64
DELAY_ENTRE_LOTES = 10

# --- 2. CONFIGURANDO MODELOS E CLIENTES ---
print("Configurando modelos e clientes...")
embed_model = CohereEmbedding(
    cohere_api_key=COHERE_API_KEY,
    model_name="embed-multilingual-v3.0",
)
llm = Cohere(api_key=COHERE_API_KEY, model="command-r")
Settings.llm = llm
Settings.embed_model = embed_model

pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY)
pinecone_index = pinecone_client.Index(INDEX_NAME)
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
print("Conexão com Pinecone estabelecida.")

# --- ETAPA DE LIMPEZA ADICIONADA ---
try:
    stats_before = pinecone_index.describe_index_stats()
    vector_count = stats_before.get('total_vector_count', 0)
    print(f"\nEstatísticas ANTES da limpeza: {vector_count} vetores.")
    
    if vector_count > 0:
        print("Limpando todos os vetores existentes no índice para uma nova indexação...")
        pinecone_index.delete(delete_all=True)
        print("Índice limpo com sucesso.")
        # Aguarda um pouco para garantir que a operação de delete seja refletida
        time.sleep(5) 
    else:
        print("Índice já está vazio. Nenhum vetor para limpar.")

    stats_after = pinecone_index.describe_index_stats()
    print(f"Estatísticas DEPOIS da limpeza: {stats_after.get('total_vector_count', 0)} vetores.")

except Exception as e:
    print(f"Erro durante a limpeza do índice: {e}")
    # Decide se quer parar ou continuar mesmo se a limpeza falhar
    # exit() # Descomente esta linha se preferir parar o script em caso de erro na limpeza


# --- 3. CARREGANDO E PROCESSANDO OS DOCUMENTOS ---
print(f"\nLendo documentos da pasta '{PDF_FOLDER_PATH}'...")
documents = SimpleDirectoryReader(PDF_FOLDER_PATH).load_data()
print(f"{len(documents)} documento(s) lido(s).")

node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=50)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)
print(f"Total de {len(nodes)} chunks (Nodes) gerados a partir dos documentos.")

# --- 4. ENVIANDO OS NODES PARA O PINECONE EM LOTES ---
print(f"\nIniciando o envio de {len(nodes)} nodes para o Pinecone em lotes de {BATCH_SIZE}...")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=[], storage_context=storage_context)

for i in tqdm(range(0, len(nodes), BATCH_SIZE), desc="Enviando lotes para o Pinecone"):
    nodes_batch = nodes[i:i + BATCH_SIZE]
    
    try:
        index.insert_nodes(nodes_batch)
        
        if i + BATCH_SIZE < len(nodes):
             time.sleep(DELAY_ENTRE_LOTES)

    except Exception as e:
        print(f"\nOcorreu um erro ao processar o lote que começa no índice {i}: {e}")
        if "TooManyRequestsError" in str(e) or "rate limit" in str(e):
            print("Rate limit atingido. Aguardando 60 segundos antes de continuar...")
            time.sleep(60)
        else:
            print("Erro não relacionado a rate limit. Pulando para o próximo lote.")
            continue


print("\nIndexação em lotes concluída!")
stats = pinecone_index.describe_index_stats()
print(f"Estatísticas do índice: {stats.get('total_vector_count', 0)} vetores no total.")


Configurando modelos e clientes...
Conexão com Pinecone estabelecida.

Estatísticas ANTES da limpeza: 0 vetores.
Índice já está vazio. Nenhum vetor para limpar.
Estatísticas DEPOIS da limpeza: 0 vetores.

Lendo documentos da pasta './documentos'...
436 documento(s) lido(s).


Parsing nodes: 100%|██████████| 436/436 [00:00<00:00, 461.38it/s]


Total de 827 chunks (Nodes) gerados a partir dos documentos.

Iniciando o envio de 827 nodes para o Pinecone em lotes de 64...


2025-08-23 19:17:32,364 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:32,578 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:32,771 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:32,972 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:33,176 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:33,405 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:33,551 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"


Upserted vectors: 100%|██████████| 64/64 [00:00<00:00, 130.27it/s]
2025-08-23 19:17:44,289 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:17:44,500 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19


Ocorreu um erro ao processar o lote que começa no índice 448: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '5de0681eae96a9229551a6338a8599cf', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '67', 'date': 'Sat, 23 Aug 2025 19:18:57 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '43', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': '6ff43108-b3f9-4517-bbcc-34a10fe4944f', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Rate limit atingido. Aguardando 60 segundos antes de continuar...


2025-08-23 19:19:57,856 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:19:58,074 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:19:58,307 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:19:58,532 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:19:58,739 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:19:58,941 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:19:59,089 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"


Upserted vectors: 100%|██████████| 64/64 [00:00<00:00, 95.34it/s]
2025-08-23 19:20:09,957 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:20:10,152 - INFO - HTTP Request: POST https://api.cohere.com/v2/embed "HTTP/1.1 200 OK"
2025-08-23 19:


Indexação em lotes concluída!
Estatísticas do índice: 763 vetores no total.
